In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [21]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
num_epochs=4
batch_size=4
lr=0.001

In [23]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [24]:
train_dataset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [25]:
trainloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
testloader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [26]:
firstdata=iter(trainloader)
features,labels=next(firstdata)
print(features.shape,labels.shape)

torch.Size([4, 3, 32, 32]) torch.Size([4])


In [27]:
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [28]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1=nn.Conv2d(3,6,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)


  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,16*5*5)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x



In [29]:
model=ConvNet().to(device)

In [30]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=lr)


In [31]:
n_total_steps=len(trainloader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(trainloader):
    images=images.to(device)
    labels=labels.to(device)


    outputs=model(images)
    loss=criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%200==0:
      print(f'Epoch: {epoch}/{num_epochs}   step: {i+1}/{n_total_steps}  loss: {loss.item():.3f} ')


Epoch: 0/4   step: 200/12500  loss: 2.322 
Epoch: 0/4   step: 400/12500  loss: 2.276 
Epoch: 0/4   step: 600/12500  loss: 2.319 
Epoch: 0/4   step: 800/12500  loss: 2.302 
Epoch: 0/4   step: 1000/12500  loss: 2.336 
Epoch: 0/4   step: 1200/12500  loss: 2.301 
Epoch: 0/4   step: 1400/12500  loss: 2.318 
Epoch: 0/4   step: 1600/12500  loss: 2.314 
Epoch: 0/4   step: 1800/12500  loss: 2.290 
Epoch: 0/4   step: 2000/12500  loss: 2.287 
Epoch: 0/4   step: 2200/12500  loss: 2.289 
Epoch: 0/4   step: 2400/12500  loss: 2.279 
Epoch: 0/4   step: 2600/12500  loss: 2.288 
Epoch: 0/4   step: 2800/12500  loss: 2.310 
Epoch: 0/4   step: 3000/12500  loss: 2.293 
Epoch: 0/4   step: 3200/12500  loss: 2.291 
Epoch: 0/4   step: 3400/12500  loss: 2.321 
Epoch: 0/4   step: 3600/12500  loss: 2.334 
Epoch: 0/4   step: 3800/12500  loss: 2.296 
Epoch: 0/4   step: 4000/12500  loss: 2.321 
Epoch: 0/4   step: 4200/12500  loss: 2.286 
Epoch: 0/4   step: 4400/12500  loss: 2.284 
Epoch: 0/4   step: 4600/12500  loss:

In [39]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for _ in range(10)]
    n_class_sample = [0 for _ in range(10)]

    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(labels.size(0)):
            label = labels[i].item()
            pred = predicted[i].item()


            if 0 <= label < 10:
                if label == pred:
                    n_class_correct[label] += 1
                n_class_sample[label] += 1
            else:
                print(f'Warning: Label {label} is out of range. Expected range: 0 to 9')

    acc = 100.0 * n_correct / n_samples if n_samples > 0 else 0.0
    print(f'Overall Accuracy: {acc:.2f}%')

    for i in range(10):
        if n_class_sample[i] > 0:
            class_acc = 100 * n_class_correct[i] / n_class_sample[i]
            print(f'Accuracy of {classes[i]}: {class_acc:.2f}%')
        else:
            print(f'Accuracy of {classes[i]}: N/A (no samples)')


Overall Accuracy: 45.95%
Accuracy of plane: 45.30%
Accuracy of car: 57.60%
Accuracy of bird: 37.50%
Accuracy of cat: 28.10%
Accuracy of deer: 27.30%
Accuracy of dog: 29.00%
Accuracy of frog: 55.70%
Accuracy of horse: 57.90%
Accuracy of ship: 66.50%
Accuracy of truck: 54.60%
